In [31]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.optimize import AdamOptimizer, QNGOptimizer
from pennylane import numpy as np

# import torch as th
# import torch.nn as nn

#from qiskit.extensions import UnitaryGate
#from qiskit import QuantumCircuit
#import qiskit.quantum_info as qi

import math

from tqdm import tqdm

from tqdm.auto import trange

In [37]:
dataset = pd.read_csv("dataset.csv")
dataset = dataset[:200]

dataset_ent = dataset[dataset.iloc[:, -1] == 0]
dataset_sep = dataset[dataset.iloc[:, -1] == 1]

In [40]:
# Drop the 17th column
X = dataset.drop(columns=dataset.columns[16])

# Separate features (X) and labels (y)
y = dataset.iloc[:, 16]  # Assuming the label is in the 17th column (index 16)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy(dtype=np.csingle)
X_test = X_test.to_numpy(dtype=np.csingle)
y_train = y_train.to_numpy().astype(int)
y_test = y_test.to_numpy().astype(int)

# Print the shapes of the resulting sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)



Shape of X_train: (800, 16)
Shape of X_test: (200, 16)
Shape of y_train: (800,)
Shape of y_test: (200,)


## QSVM

In [26]:
def statepreparation(data):
    data = np.array(data).reshape(4, 4)
    qml.QubitUnitary(data,wires=[0,1])

In [27]:
num_qubits = 2

dev_kernel = qml.device("lightning.qubit", wires=num_qubits)

projector = np.zeros((2**num_qubits, 2**num_qubits))
projector[0, 0] = 1


@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    x1 = np.array(x1).reshape(4, 4)
    x2 = np.array(x2).reshape(4, 4)

    qml.QubitUnitary(x1, wires=range(num_qubits))
    qml.adjoint(qml.QubitUnitary)(x2, wires=range(num_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(num_qubits)))

/Users/paolodarold/opt/anaconda3/envs/QML/lib/python3.9/site-packages/pennylane_lightning/lightning_qubit/lightning_qubit.py:901: UserWarning: Pre-compiled binaries for lightning.qubit are not available. Falling back to using the Python-based default.qubit implementation. To manually compile from source, follow the instructions at https://pennylane-lightning.readthedocs.io/en/latest/installation.html.
  warn(


In [28]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [41]:
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [42]:
predictions_train = svm.predict(X_train)
accuracy_score(predictions_train, y_train)

0.53125

In [43]:
predictions_test = svm.predict(X_test)
accuracy_score(predictions_test, y_test)

0.495